In [1]:
import pandas as pd 
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgbm
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split, ParameterSampler, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
import scipy

In [2]:
df = pd.read_csv("train.csv")
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [3]:
df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [23]:
df = df.drop(['casual','registered'], axis=1)
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,1
...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,129


In [24]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [25]:
# df['day'] = df['datetime'].dt.day # retirer car il s'arrete jusqu'au 19
df['hours'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month
df['week'] = df['datetime'].dt.day_name()
df = df.drop(['datetime','atemp'], axis=1)

In [26]:
df

,season,holiday,workingday,weather,temp,humidity,windspeed,count,hours,month,week
0,1,0,0,1,9.84,81,0.0000,16,0,1,Saturday
1,1,0,0,1,9.02,80,0.0000,40,1,1,Saturday
2,1,0,0,1,9.02,80,0.0000,32,2,1,Saturday
3,1,0,0,1,9.84,75,0.0000,13,3,1,Saturday
4,1,0,0,1,9.84,75,0.0000,1,4,1,Saturday
...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,50,26.0027,336,19,12,Wednesday
10882,4,0,1,1,14.76,57,15.0013,241,20,12,Wednesday
10883,4,0,1,1,13.94,61,15.0013,168,21,12,Wednesday
10884,4,0,1,1,13.94,61,6.0032,129,22,12,Wednesday


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   humidity    10886 non-null  int64  
 6   windspeed   10886 non-null  float64
 7   count       10886 non-null  int64  
 8   hours       10886 non-null  int64  
 9   month       10886 non-null  int64  
 10  week        10886 non-null  object 
dtypes: float64(2), int64(8), object(1)
memory usage: 935.6+ KB


In [28]:
y = df['count']

df = df.drop(['count'], axis=1)
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [29]:
gridParams = {
    'learning_rate': [0.05, 1],
    'num_leaves': range(1,100),
    'max_depth' : range(1,100),
    'colsample_bytree' : [0.1,0.5,0.7,1],
    'subsample' : [0.1,0.2,0.3,0.4,0.5,0.7,0.8,0.9],
    'min_split_gain' : [0.1,0.2,0.3,0.4,0.5,0.7],

    }

In [30]:
list(np.arange(0.01,1,0.1))

[0.01,
 0.11,
 0.21000000000000002,
 0.31000000000000005,
 0.41000000000000003,
 0.51,
 0.6100000000000001,
 0.7100000000000001,
 0.81,
 0.91]

In [31]:
num_attributes = ['hours', 'month','humidity', 'temp', 'windspeed']
cat_attributes = ['season', 'weather','workingday', 'holiday','week']

# season	holiday	workingday	weather	temp	humidity	windspeed	hours	month	week

processor = make_column_transformer((StandardScaler(),num_attributes),(OneHotEncoder(),cat_attributes))

In [32]:
X_train_transform=processor.fit_transform(X_train)

In [33]:
#modelling
clf = lgbm.LGBMRegressor()
grid = RandomizedSearchCV(clf,gridParams,verbose=1,cv=5,n_jobs = -1,n_iter=10, scoring='r2')
grid.fit(X_train_transform,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=LGBMRegressor(), n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 0.5, 0.7, 1],
                                        'learning_rate': [0.05, 1],
                                        'max_depth': range(1, 100),
                                        'min_split_gain': [0.1, 0.2, 0.3, 0.4,
                                                           0.5, 0.7],
                                        'num_leaves': range(1, 100),
                                        'subsample': [0.1, 0.2, 0.3, 0.4, 0.5,
                                                      0.7, 0.8, 0.9]},
                   scoring='r2', verbose=1)

In [34]:
#best parameters
grid.best_params_

{'subsample': 0.8,
 'num_leaves': 62,
 'min_split_gain': 0.1,
 'max_depth': 87,
 'learning_rate': 0.05,
 'colsample_bytree': 0.7}

In [35]:
gridParams = {
    'learning_rate': [0.01,0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1],
    'num_leaves': range(70,90),
    'max_depth' : range(40,50),
    'colsample_bytree' : [1],
    'subsample' : [0.9],
    'min_split_gain' : [0.1],
    }

In [36]:
grid = GridSearchCV(clf,gridParams,verbose=1,cv=5)


In [37]:
grid.fit(X_train_transform,y_train)


Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'colsample_bytree': [1],
                         'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                           0.07, 0.08, 0.09, 0.1],
                         'max_depth': range(40, 50), 'min_split_gain': [0.1],
                         'num_leaves': range(70, 90), 'subsample': [0.9]},
             verbose=1)

In [38]:
grid.best_params_


{'colsample_bytree': 1,
 'learning_rate': 0.1,
 'max_depth': 40,
 'min_split_gain': 0.1,
 'num_leaves': 80,
 'subsample': 0.9}

In [39]:
#modelling
clf = lgbm.LGBMRegressor(
subsample= 0.9,
num_leaves= 80,
min_split_gain= 0.1,
max_depth= 40,
learning_rate= 0.1,
colsample_bytree= 1)
model = make_pipeline(processor, clf) 
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['hours', 'month', 'humidity',
                                                   'temp', 'windspeed']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['season', 'weather',
                                                   'workingday', 'holiday',
                                                   'week'])])),
                ('lgbmregressor',
                 LGBMRegressor(colsample_bytree=1, max_depth=40,
                               min_split_gain=0.1, num_leaves=80,
                               subsample=0.9))])

In [40]:
#Prediction
y_pred = model.predict(X_test)

In [41]:
model.score(X_test, y_test)

0.8856697927593243

In [42]:
import pickle

In [43]:
with open('objet.pkl','wb'): #dump va écrire des bytes pas des str
  pickle.dump(model,f)

In [44]:
# pickle.dump(model, 'objet.pkl','w')

In [45]:
from sklearn.model_selection import cross_val_score

In [46]:
cross_val_score(model, X_train, y_train, cv=200)

C:\Users\USER\anaconda3\envs\lightgbm\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\envs\lightgbm\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\USER\anaconda3\envs\lightgbm\lib\site-packages\sklearn\metrics\_scorer.py", line 105, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "C:\Users\USER\anaconda3\envs\lightgbm\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\USER\anaconda3\envs\lightgbm\lib\site-packages\sklearn\utils\metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "C:\Users\USER\anaconda3\envs\lightgbm\lib\

array([0.94385349, 0.89515533, 0.87751868, 0.91207761, 0.92639379,
       0.81421032, 0.82500369, 0.89336691, 0.81321744, 0.92488386,
       0.81093786, 0.91423621, 0.92947183, 0.90508437, 0.92695334,
       0.87741884, 0.71905127, 0.9251887 , 0.90796902, 0.88464405,
       0.92071809, 0.87964652, 0.90511643, 0.86446338, 0.90090156,
       0.89260377, 0.81868255, 0.86945432, 0.95551625, 0.82551617,
       0.90435971, 0.75940789, 0.84903461, 0.84017792, 0.91552297,
       0.93136174, 0.90561803, 0.86927458, 0.92083623, 0.88639876,
       0.85837904, 0.85164988, 0.87783123, 0.861329  , 0.8690668 ,
       0.84425379, 0.89909047, 0.74340425, 0.88574055, 0.87867832,
       0.91106924, 0.87148076, 0.86501504, 0.89848743, 0.8800988 ,
       0.8019471 , 0.87487356, 0.85291028, 0.83987542, 0.9210837 ,
       0.93627606, 0.92683449, 0.91638845, 0.73023062, 0.90975143,
       0.89106235, 0.89557379, 0.83048474, 0.91038196, 0.85992758,
       0.87372612, 0.88453976, 0.93744674, 0.8530475 , 0.86441

In [47]:
# lgbm_tuned = grid.best_estimator_

In [48]:
# r2_tuned = grid.best_params_
# r2_regular = r2_score(y_train, reg.predict(X_train))

In [49]:
# expected_y  = y_test
# # predicted_y = reg.predict(X_test)

In [50]:
# print(metrics.r2_score(expected_y, predicted_y))
# print(metrics.mean_squared_log_error(expected_y, predicted_y))

In [51]:
# plt.figure(figsize=(10,10))
# sns.regplot(expected_y, predicted_y, fit_reg=True, scatter_kws={"s": 100})

In [52]:
# lgbm.plot_importance(reg)